In [28]:
import sys
import os

# Obtiene la ruta absoluta a la raíz del proyecto (ajusta según tu estructura)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [44]:
%load_ext autoreload
%autoreload 2

from src.predict import cargar_modelo, predecir, evaluar, evaluar_meses_tabla

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import pandas as pd
import joblib
from sklearn.metrics import f1_score

# Cargamos el modelo y las funciones que necesitamos del notebook 00

In [31]:
rfc = joblib.load("./model/random_forest.joblib")

In [32]:
numeric_feat = [
"pickup_weekday",
"pickup_hour",
'work_hours',
"pickup_minute",
"passenger_count",
'trip_distance',
'trip_time',
'trip_speed'
]
categorical_feat = [
    "PULocationID",
    "DOLocationID",
    "RatecodeID",
]
EPS = 1e-7
features = numeric_feat + categorical_feat
target_col = "high_tip"

In [33]:
def preprocess(df, target_col):

   # Basic cleaning
    df = df[df['fare_amount'] > 0].reset_index(drop=True)  # avoid divide-by-zero
    # add target
    df['tip_fraction'] = df['tip_amount'] / df['fare_amount']
    df[target_col] = df['tip_fraction'] > 0.2

    # add features
    df['pickup_weekday'] = df['tpep_pickup_datetime'].dt.weekday
    df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
    df['pickup_minute'] = df['tpep_pickup_datetime'].dt.minute
    df['work_hours'] = (df['pickup_weekday'] >= 0) & (df['pickup_weekday'] <= 4) & (df['pickup_hour'] >= 8) & (df['pickup_hour'] <= 18)
    df['trip_time'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.seconds
    df['trip_speed'] = df['trip_distance'] / (df['trip_time'] + EPS)

    # drop unused columns
    df = df[['tpep_dropoff_datetime'] + features + [target_col]]
    df[features + [target_col]] = df[features + [target_col]].astype("float32").fillna(-1.0)

    # convert target to int32 for efficiency (it's just 0s and 1s)
    df[target_col] = df[target_col].astype("int32")

    return df.reset_index(drop=True)

# Evaluación Enero 2020

In [34]:
taxi_march= pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-01.parquet')
taxi_test = preprocess(taxi_march, target_col=target_col).head(1000)
taxi_test.head()

preds_test = rfc.predict_proba(taxi_test[features])
preds_test_labels = [p[1] for p in preds_test.round()]
print(f'enero-2020 F1: {f1_score(taxi_test[target_col], preds_test_labels)}')

enero-2020 F1: 0.7074542897327707


In [35]:
taxi_test.to_csv('./app/data/yellow_tripdata_2020-01_test.csv', index=False)

# Evaluación Febrero 2020

In [36]:
taxi_march= pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-02.parquet')
taxi_test = preprocess(taxi_march, target_col=target_col).head(1000)
taxi_test.head()

preds_test = rfc.predict_proba(taxi_test[features])
preds_test_labels = [p[1] for p in preds_test.round()]
print(f'febrero-2020 F1: {f1_score(taxi_test[target_col], preds_test_labels)}')

febrero-2020 F1: 0.7456492637215528


In [37]:
taxi_test.to_csv('./app/data/yellow_tripdata_2020-02_test.csv', index=False)

# Evaluación Marzo 2020

In [38]:
taxi_march= pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-03.parquet')
taxi_test = preprocess(taxi_march, target_col=target_col).head(1000)
taxi_test.head()

preds_test = rfc.predict_proba(taxi_test[features])
preds_test_labels = [p[1] for p in preds_test.round()]
print(f'marzo-2020 F1: {f1_score(taxi_test[target_col], preds_test_labels)}')

marzo-2020 F1: 0.7408880053015242


In [39]:
taxi_test.to_csv('./app/data/yellow_tripdata_2020-03_test.csv', index=False)

# Evaluación Abril 2020

In [40]:
taxi_may= pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-04.parquet')
taxi_test = preprocess(taxi_may, target_col=target_col).head(1000)
taxi_test.head()

preds_test = rfc.predict_proba(taxi_test[features])
preds_test_labels = [p[1] for p in preds_test.round()]
print(f'abril-2020 F1: {f1_score(taxi_test[target_col], preds_test_labels)}')

abril-2020 F1: 0.5471698113207547


In [41]:
taxi_test.to_csv('./app/data/yellow_tripdata_2020-04_test.csv', index=False)

# Evaluación Mayo 2020

In [42]:
taxi_may= pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-05.parquet')
taxi_test = preprocess(taxi_may, target_col=target_col).head(1000)
taxi_test.head()

preds_test = rfc.predict_proba(taxi_test[features])
preds_test_labels = [p[1] for p in preds_test.round()]
print(f'mayo-2020 F1: {f1_score(taxi_test[target_col], preds_test_labels)}')

mayo-2020 F1: 0.5536723163841808


In [43]:
taxi_test.to_csv('./app/data/yellow_tripdata_2020-05_test.csv', index=False)

# Evaluación de varios meses

In [45]:
# Puedes especificar los meses o dejarlo vacío para evaluar todos los disponibles en config
resultados = evaluar_meses_tabla()

# Mostrar la tabla de resultados
print(resultados)

INFO:src.predict:Modelo cargado desde ./model/random_forest.joblib
c:\Users\kurtc\Documentos\GitHub\producto-datos-lab\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:src.predict:Predicciones realizadas para 996 muestras
c:\Users\kurtc\Documentos\GitHub\producto-datos-lab\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:src.predict:Predicciones realizadas para 995 muestras
c:\Users\kurtc\Documentos\GitHub\producto-datos-lab\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:src.predict:Predicciones realizadas para 994 muestras
c:\Users\kurtc\Documentos\GitHub\producto-datos-lab\.v

       mes  n_samples  f1_score
0  2020-01        996  0.708539
1  2020-02        995  0.746469
2  2020-03        994  0.742172
3  2020-04        981  0.546154
4  2020-05        989  0.553435


In [46]:
resultados

,mes,n_samples,f1_score
0,2020-01,996,0.708539
1,2020-02,995,0.746469
2,2020-03,994,0.742172
3,2020-04,981,0.546154
4,2020-05,989,0.553435
